In [87]:
%run classifiers.py --adaboost --cut_scan


deselected_features ['dsid', 'event_weight', 'lep_charge1', 'lep_charge2', 'n_bjets']

selected_features = ['dPhi_jj_ll', 'dPhi_jj_met', 'dPhi_ll_met', 'dR_jj', 'dR_ll', 'jet_eta1', 'jet_eta2', 'jet_eta3', 'jet_m1', 'jet_m2', 'jet_m3', 'jet_phi1', 'jet_phi2', 'jet_phi3', 'jet_pt1', 'jet_pt2', 'jet_pt3', 'lep_eta1', 'lep_eta2', 'lep_phi1', 'lep_phi2', 'lep_pt1', 'lep_pt2', 'met', 'met_phi', 'mjj', 'mll', 'n_jets', 'pt_jj', 'pt_ll']

n_selected_features = 30

Read in background file: ../../../ewk/hdf5_files/2L_bkg_flat_ext.h5
Read in signal file: ../../../ewk/hdf5_files/2L_sig_flat_ext.h5

class_weight_vect [1. 1.]
class_weight_dict {0: 1.0, 1: 1.0}

scale_pos_weight 1.0

before imp: np.any(X_bkg_sel_arr == -999) True
before imp: np.any(X_bkg_shuffled == -999) False
after imp: np.any(X == -999) False

# training examples:
Background  : 570
Signal      : 571

# test examples:
Background  : 282
Signal      : 281

INFO  Applying event weights to the examples during training

model.get_para

In [88]:
%run classifiers.py --xgboost --cut_scan


deselected_features ['dsid', 'event_weight', 'lep_charge1', 'lep_charge2', 'n_bjets']

selected_features = ['dPhi_jj_ll', 'dPhi_jj_met', 'dPhi_ll_met', 'dR_jj', 'dR_ll', 'jet_eta1', 'jet_eta2', 'jet_eta3', 'jet_m1', 'jet_m2', 'jet_m3', 'jet_phi1', 'jet_phi2', 'jet_phi3', 'jet_pt1', 'jet_pt2', 'jet_pt3', 'lep_eta1', 'lep_eta2', 'lep_phi1', 'lep_phi2', 'lep_pt1', 'lep_pt2', 'met', 'met_phi', 'mjj', 'mll', 'n_jets', 'pt_jj', 'pt_ll']

n_selected_features = 30

Read in background file: ../../../ewk/hdf5_files/2L_bkg_flat_ext.h5
Read in signal file: ../../../ewk/hdf5_files/2L_sig_flat_ext.h5

class_weight_vect [1. 1.]
class_weight_dict {0: 1.0, 1: 1.0}

scale_pos_weight 1.0

before imp: np.any(X_bkg_sel_arr == -999) True
before imp: np.any(X_bkg_shuffled == -999) False
after imp: np.any(X == -999) False

# training examples:
Background  : 570
Signal      : 571

# test examples:
Background  : 282
Signal      : 281

INFO  Applying event weights to the examples during training

model.get_para

/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/lib/python2.7/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/lib/python2.7/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:



----- TRAINING -----
Confusion matrix, without normalization
[[359 211]
 [  6 565]]

Normalized confusion matrix
[[0.63 0.37]
 [0.01 0.99]]

----- TESTING -----
Confusion matrix, without normalization
[[176 106]
 [ 10 271]]

Normalized confusion matrix
[[0.62 0.38]
 [0.04 0.96]]
Embedding font /eos/user/k/kvadla/.local/lib/python2.7/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf.
Writing TrueType font.

Training sample....
  Signal identified as signal (%)        :  98.9492119089
  Signal identified as background (%)    :  1.05078809107
  Background identified as signal (%)    :  37.0175438596
  Background identified as background (%):  62.9824561404

Testing sample....
  Signal identified as signal (%)        :  96.4412811388
  Signal identified as background (%)    :  3.55871886121
  Background identified as signal (%)    :  37.5886524823
  Background identified as background (%):  62.4113475177

Area under ROC =  0.8888720627949824

sum_weights_test_sig 15.1704557985067

In [89]:
%run classifiers.py --nn --cut_scan


deselected_features ['dsid', 'event_weight', 'lep_charge1', 'lep_charge2', 'n_bjets']

selected_features = ['dPhi_jj_ll', 'dPhi_jj_met', 'dPhi_ll_met', 'dR_jj', 'dR_ll', 'jet_eta1', 'jet_eta2', 'jet_eta3', 'jet_m1', 'jet_m2', 'jet_m3', 'jet_phi1', 'jet_phi2', 'jet_phi3', 'jet_pt1', 'jet_pt2', 'jet_pt3', 'lep_eta1', 'lep_eta2', 'lep_phi1', 'lep_phi2', 'lep_pt1', 'lep_pt2', 'met', 'met_phi', 'mjj', 'mll', 'n_jets', 'pt_jj', 'pt_ll']

n_selected_features = 30

Read in background file: ../../../ewk/hdf5_files/2L_bkg_flat_ext.h5
Read in signal file: ../../../ewk/hdf5_files/2L_sig_flat_ext.h5

class_weight_vect [1. 1.]
class_weight_dict {0: 1.0, 1: 1.0}

scale_pos_weight 1.0

before imp: np.any(X_bkg_sel_arr == -999) True
before imp: np.any(X_bkg_shuffled == -999) False
after imp: np.any(X == -999) False

# training examples:
Background  : 570
Signal      : 571

# test examples:
Background  : 282
Signal      : 281

INFO  Applying event weights to the examples during training

Building and t

1141/1141 [==============================] - 0s 30us/step - loss: 0.0502 - acc: 0.5092
Epoch 72/100
1141/1141 [==============================] - 0s 53us/step - loss: 0.0500 - acc: 0.5118
Epoch 73/100
1141/1141 [==============================] - 0s 29us/step - loss: 0.0499 - acc: 0.5101
Epoch 74/100
1141/1141 [==============================] - 0s 57us/step - loss: 0.0496 - acc: 0.5083
Epoch 75/100
1141/1141 [==============================] - 0s 29us/step - loss: 0.0495 - acc: 0.5101
Epoch 76/100
1141/1141 [==============================] - 0s 56us/step - loss: 0.0494 - acc: 0.5074
Epoch 77/100
1141/1141 [==============================] - 0s 31us/step - loss: 0.0492 - acc: 0.5110
Epoch 78/100
1141/1141 [==============================] - 0s 55us/step - loss: 0.0490 - acc: 0.5145
Epoch 79/100
1141/1141 [==============================] - 0s 29us/step - loss: 0.0488 - acc: 0.5110
Epoch 80/100
1141/1141 [==============================] - 0s 57us/step - loss: 0.0488 - acc: 0.5145
Epoch 81/100
